---

# Section 4.2: More Complex *TxGraffiti* Expressions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RandyRDavila/AI-discovery-in-mathematics-with-TxGraffiti/blob/main/notebooks/MoreComplexLinearInequalities.ipynb)

In this notebook we walk through the process of implementing *TxGraffiti* on a small graph dataset as shown in Section 4, and in particular demonstrate the generation of more complex linear inequalities discussed in Section 4.2. Click the above Google Colab button to open this notebook.

To begin, we import necessary dependencies.

---

In [73]:
# PuLP will be needed to solve optimization processes below.
!pip install pulp

import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
sns.set_theme()

from pulp import *
from fractions import Fraction
from itertools import combinations

----

# Build a Sample Dataset
Below we have the edge lists of 9 simple connected graphs. Namely, the 3-vertex path graph $P_3$, the 3-vertex cycle graph $C_3$, the diamond graph, the 4-vertex complete graph $K_4$, the complete bipartite graph $K_{4, 4}$, the star graph $K_{1, 3}$, the double star graph $S(2, 2)$, and the graph obtained by attaching two triangles by a single edge.

----

In [2]:
edge_list_1 = [(0, 1), (1, 2)]
edge_list_2 = [(0, 1), (0, 2), (1, 2)]
edge_list_3 = [(0, 1), (0, 3), (1, 2), (2, 3)]
edge_list_4 = [(0, 1), (0, 2), (0, 3), (1, 2), (2, 3)]
edge_list_5 = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
edge_list_6 = [
    (0, 4), (0, 5), (0, 6), (0, 7),
    (1, 4), (1, 5), (1, 6), (1, 7),
    (2, 4), (2, 5), (2, 6), (2, 7),
    (3, 4), (3, 5), (3, 6), (3, 7),
]
edge_list_7 = [(0, 1), (0, 2), (0, 3)]
edge_list_8 = [(0, 1), (0, 2), (0, 3), (3, 4), (3, 5)]
edge_list_9 = [(0, 1), (0, 2), (0, 3), (1, 2), (3, 4), (3, 5), (4, 5)]

dataset = [
    edge_list_1,
    edge_list_2,
    edge_list_3,
    edge_list_4,
    edge_list_5,
    edge_list_6,
    edge_list_7,
    edge_list_8,
    edge_list_9,
]

names = [
    "G_1",
    "G_2",
    "G_3",
    "G_4",
    "G_5",
    "G_6",
    "G_7",
    "G_8",
    "G_9",
]

----

## Choose a Collection of Properties to Compute

In the following code cell we write functions to compute several graph invariants. For simplicity, we choose the following simple numerical invariants:

* The *order* of $G$, denoted $n(G)$ is the number of vertices in $G$.
* The *matching number* of $G$, denoted $\mu(G)$, is the cardinality of a maximum set of edges that do not share endpoints.
* The *independence number* of $G$, denoted $\alpha(G)$, is the cardinality of a maximum set of pairwise nonadjacent vertices.

We will also use the following boolean properities of a graph:
* A graph is *connected* if there exists a path between any two vertices in $G$.
* A graph is a *tree* if it is connected and does not contain a cycle as a subgraph.
* A graph is connected and *regular* if it is connected and every vertex degree is the same.
* A graph is connected and *bipartite* if it is connected and the vertex set can be partitioned into two disjoint independent sets.

To make conjectures involving two or more invariants and also nonlinear conjectures, we also include functions for computing the following:
* The *maximum degree squared*, denoted $\Delta(G)^2$, is the square of the maximum degree of $G$.


----

In [82]:
# The order of the graph - the number of vertices.
def n(G):
    return G.number_of_nodes()

# The matching number of a graph - the cardinality of a maximum set of edges
# which do not share an endpoint.
def matching_number(G):
    prob = LpProblem("MaximumMatchingSet", LpMaximize)
    variables = {edge: LpVariable("x{}".format(i + 1), 0, 1, LpBinary) for i, edge in enumerate(G.edges())}

    # Set the maximum matching objective function
    prob += lpSum(variables)

    # Set constraints
    for node in G.nodes():
        incident_edges = [variables[edge] for edge in variables if node in edge]
        prob += sum(incident_edges) <= 1

    prob.solve()
    solution_set = {edge for edge in variables if variables[edge].value() == 1}
    return len(solution_set)

# The independence number - the cardinality of a maximum set of pairwise nonadjacent
# vertices.
def independence_number(G):
    prob = LpProblem("MaximumIndependentSet", LpMaximize)
    variables = {node: LpVariable("x{}".format(i + 1), 0, 1, LpBinary) for i, node in enumerate(G.nodes())}

    # Set the domination number objective function
    prob += lpSum(variables)

    # Set constraints for independence
    for e in G.edges():
        prob += variables[e[0]] + variables[e[1]] <= 1

    prob.solve()
    solution_set = {node for node in variables if variables[node].value() == 1}
    return len(solution_set)


# The minimum vertex degree of the graph.
def minimum_degree(G):
    return min(G.degree(node) for node in G.nodes())

# The maximum vertex degree of the graph
def maximum_degree(G):
    return max(G.degree(node) for node in G.nodes())


# The maximum degree squared.
def maximum_degree_squared(G):
    return maximum_degree(G) ** 2

# Is the graph connected?
def is_connected(G):
    return nx.is_connected(G)

# Is the graph a tree?
def is_tree(G):
    return nx.is_tree(G)

# Is the graph regular?
def is_regular(G):
    return minimum_degree(G) == maximum_degree(G)

# Is the graph bipartite?
def is_bipartite(G):
    return nx.is_bipartite(G)

---

## Create a Tabular Dataset

TxGraffiti produces conjectures one a given tabular dataset like a Pandas DataFrame. So, our next step is to populate a DataFrame with the desired functions applied to the mathematical objects in question - each row being an instance of an object and each column being a function applied to said object.

---

In [75]:
# Create a DataFrame to store the results
rows = []
for i, edge_list in enumerate(dataset):
    G = nx.Graph()
    G.add_edges_from(edge_list)

    row = {
        "name": names[i],
        "n": n(G),
        "matching_number": matching_number(G),
        "independence_number": independence_number(G),
        "minimum_degree": minimum_degree(G),
        "maximum_degree": maximum_degree(G),
        "maximum_degree_squared": maximum_degree_squared(G),
        "connected": is_connected(G),
        "tree": is_tree(G),
        "connected and regular": is_regular(G),
        "connected and bipartite": is_bipartite(G),
    }
    rows.append(row)

# Convert the list of rows into a pandas DataFrame
df = pd.DataFrame(rows)

# Show the DataFrame
df

,name,n,matching_number,independence_number,minimum_degree,maximum_degree,maximum_degree_squared,connected,tree,connected and regular,connected and bipartite
0,G_1,3,1,2,1,2,4,True,True,False,True
1,G_2,3,1,1,2,2,4,True,False,True,False
2,G_3,4,2,2,2,2,4,True,False,True,True
3,G_4,4,2,2,2,3,9,True,False,False,False
4,G_5,4,2,1,3,3,9,True,False,True,False
5,G_6,8,4,4,4,4,16,True,False,True,True
6,G_7,4,1,3,1,3,9,True,True,False,True
7,G_8,6,2,4,1,3,9,True,True,False,True
8,G_9,6,3,2,2,3,9,True,False,False,False


---

## Custome Conjecture Objects
For convience we now write several Python classes as containers for the data stored in conjectures. The primary reason for implementing these classes is to store the hypothesis, conclusion, and object data associated with each found conjecture.

---

In [102]:
 class Hypothesis:
    """
    A class for graph hypotheses.

    Attributes
    ----------
    statement : string
        The statement of the hypothesis.

    Methods
    -------
    __str__():
        Returns the statement of the hypothesis.
    __repr__():
        Returns the statement of the hypothesis.
    __call__(name, df):
        Returns the value of the hypothesis for the graph with the given name in
        the given dataframe.
    """
    def __init__(self, statement):
        self.statement = statement

    def __str__(self):
        return f"{self.statement}"

    def __repr__(self):
        return f"{self.statement}"

    def __call__(self, name, df):
        return df.loc[df["name"] == f"{name}.txt"][self.statement]


class MultiLinearConclusion:
    """
    A class for multilinear graph conclusions involving two or more invariants.

    Attributes
    ----------
    lhs : string
        The left-hand side of the conclusion.
    inequality : string
        The inequality of the conclusion.
    slopes : list of floats
        The slopes of the conclusion (one for each invariant).
    rhs : list of strings
        The right-hand sides of the conclusion (one for each invariant).
    intercept : float
        The intercept of the conclusion.

    Methods
    -------
    __str__():
        Returns the conclusion as a string with better formatting for 1s, -1s, and 0s.
    __repr__():
        Returns the conclusion as a string.
    __eq__(other):
        Returns True if the conclusion is equal to the other conclusion, and
        False otherwise.
    __ne__(other):
        Returns True if the conclusion is not equal to the other conclusion, and
        False otherwise.
    __call__(name, df):
        Returns the value of the conclusion for the graph with the given name in
        the given dataframe.
    """
    def __init__(self, lhs, inequality, slopes, rhs, intercept):
        self.lhs = lhs
        self.inequality = inequality
        self.slopes = slopes
        self.rhs = rhs
        self.intercept = intercept

    def __str__(self):
        slope_terms = []
        for m, rhs in zip(self.slopes, self.rhs):
            if m == 1:
                slope_terms.append(f"{rhs}")
            elif m == -1:
                slope_terms.append(f"- {rhs}")
            elif m != 0:
                slope_terms.append(f"{m} * {rhs}")

        # Join slope terms appropriately
        slope_str = " + ".join(slope_terms)

        # Format intercept
        if self.intercept > 0:
            result = f"{slope_str} + {self.intercept}"
        elif self.intercept < 0:
            result = f"{slope_str} - {abs(self.intercept)}"
        else:
            result = slope_str

        # Clean up + - signs and unnecessary spaces
        result = result.replace("+ -", "- ").strip()

        # Return the final formatted string
        return f"{self.lhs} {self.inequality} {result}"

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return (
            self.lhs == other.lhs and
            self.inequality == other.inequality and
            self.slopes == other.slopes and
            self.rhs == other.rhs and
            self.intercept == other.intercept
        )

    def __ne__(self, other):
        return not self.__eq__(other)

    def __call__(self, name, df):
        # Evaluate the linear inequality for the given graph in the dataframe.
        data = df.loc[df["name"] == f"{name}"]
        rhs_value = sum(m * data[r].values[0] for m, r in zip(self.slopes, self.rhs)) + self.intercept
        if self.inequality == "<=":
            return data[self.lhs].values[0] <= rhs_value
        else:
            return data[self.lhs].values[0] >= rhs_value

    def __hash__(self):
        return hash((self.lhs, self.inequality, tuple(self.slopes), tuple(self.rhs), self.intercept))




class LinearConjecture:
    """
    A class for linear or multi-linear graph conjectures.

    Attributes
    ----------
    conclusion : MultiLinearConclusion
        The conclusion of the conjecture (now supporting multiple invariants).
    hypothesis : Hypothesis
        The hypothesis of the conjecture.
    symbol : string
        The symbol of the conjecture.
    touch : int
        The number of graphs that touch the conjecture, i.e., the number of graphs
        that satisfy the hypothesis and the conclusion with equality.

    Methods
    -------
    __repr__():
        Returns the conjecture as a string.
    __call__(name, df):
        Returns the value of the conjecture for the graph with the given name in
        the given dataframe.
    __eq__(other):
        Returns True if the conjecture is equal to the other conjecture, and
        False otherwise.
    get_sharp_graphs(df):
        Returns the graphs that touch the conjecture.
    """
    def __init__(self, hypothesis, conclusion, symbol="G", touch=0):
        self.hypothesis = hypothesis
        self.conclusion = conclusion
        self.symbol = symbol
        self.touch = touch

    def __repr__(self):
        hypothesis = f"If {self.symbol} is {self.hypothesis}"
        return f"{hypothesis}, then {self.conclusion}"

    def __call__(self, name, df):
        if self.hypothesis(name, df).values[0]:
            return self.conclusion(name, df)
        else:
            return False

    def __hash__(self):
        return hash((self.hypothesis, self.conclusion, self.symbol))

    def __eq__(self, other):
        return self.hypothesis == other.hypothesis and self.conclusion == other.conclusion and self.symbol == other.symbol

    def get_sharp_graphs(self, df):
        return df.loc[(df[self.hypothesis.statement] == True) &
                      (df[self.conclusion.lhs] == sum(self.conclusion.slopes[i] * df[self.conclusion.rhs[i]]
                                                      for i in range(len(self.conclusion.rhs))) + self.conclusion.intercept)]




---

# Generating a Single Inequality

Our first goal will be to formulate a single upper bound inequality for a target invariant in terms of **two other invariants**. To do this, we solve the following linear optimization problem whose feasible solutions form such upper bounds on a target invariant.

---

In [103]:
def make_upper_linear_conjecture_two_vars(
        df,
        target,
        other1,
        other2,
        hyp="connected",
        symbol="G"
    ):
    """
    Returns a LinearConjecture object with the given hypothesis, target, and two other variables.
    The conclusion is determined by solving a linear program. The inequality is <=.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the data.
    target : string
        The name of the target variable.
    other1 : string
        The name of the first other variable.
    other2 : string
        The name of the second other variable.
    hyp : string
        The name of the hypothesis variable.
    symbol : string
        The symbol of the object in the conjecture.

    Returns
    -------
    LinearConjecture
        The conjecture with the given hypothesis, target, and two other variables.
    """

    # Filter data for the hypothesis condition.
    df = df[df[hyp] == True]

    # Extract the data from the dataframe as individual lists.
    X1 = df[other1].tolist()  # List of values for first invariant
    X2 = df[other2].tolist()  # List of values for second invariant
    Y = df[target].tolist()   # List of values for the target variable

    # Initialize the LP problem.
    prob = LpProblem("Test_Problem", LpMinimize)

    # Initialize the variables for the LP.
    w1 = LpVariable("w1")
    w2 = LpVariable("w2")
    b = LpVariable("b")

    # Define the objective function.
    prob += lpSum([w1 * x1 + w2 * x2 + b - y for x1, x2, y in zip(X1, X2, Y)])

    # Define the LP constraints.
    for x1, x2, y in zip(X1, X2, Y):
        prob += w1 * x1 + w2 * x2 + b >= y

    # Solve the LP.
    prob.solve()

    # Extract the solution.
    m1 = Fraction(w1.varValue).limit_denominator(10)
    m2 = Fraction(w2.varValue).limit_denominator(10)
    b_value = Fraction(b.varValue).limit_denominator(10)

    # Compute the number of instances of equality - the touch number of the conjecture.
    touch = sum(1 for x1, x2, y in zip(X1, X2, Y) if y == m1 * x1 + m2 * x2 + b_value)

    # Create the hypothesis and conclusion objects.
    hypothesis = Hypothesis(hyp)
    conclusion = MultiLinearConclusion(target, "<=", [m1, m2], [other1, other2], b_value)

    # Return the full conjecture object (not the conclusion directly).
    return LinearConjecture(hypothesis, conclusion, symbol, touch)

def make_lower_linear_conjecture_two_vars(
        df,
        target,
        other1,
        other2,
        hyp="connected",
        symbol="G"
    ):

    # Filter data for the hypothesis condition.
    df = df[df[hyp] == True]

    # Extract the data from the dataframe as individual lists.
    X1 = df[other1].tolist()  # List of values for first invariant
    X2 = df[other2].tolist()  # List of values for second invariant
    Y = df[target].tolist()   # List of values for the target variable

    # Initialize the LP problem.
    prob = LpProblem("Test_Problem", LpMaximize)

    # Initialize the variables for the LP.
    w1 = LpVariable("w1")
    w2 = LpVariable("w2")
    b = LpVariable("b")

    # Define the objective function.
    prob += lpSum([w1 * x1 + w2 * x2 + b - y for x1, x2, y in zip(X1, X2, Y)])

    # Define the LP constraints.
    for x1, x2, y in zip(X1, X2, Y):
        prob += w1 * x1 + w2 * x2 + b <= y

    # Solve the LP.
    prob.solve()

    # Extract the solution.
    m1 = Fraction(w1.varValue).limit_denominator(10)
    m2 = Fraction(w2.varValue).limit_denominator(10)
    b_value = Fraction(b.varValue).limit_denominator(10)

    # Compute the number of instances of equality - the touch number of the conjecture.
    touch = sum(1 for x1, x2, y in zip(X1, X2, Y) if y == m1 * x1 + m2 * x2 + b_value)

    # Create the hypothesis and conclusion objects.
    hypothesis = Hypothesis(hyp)
    conclusion = MultiLinearConclusion(target, ">=", [m1, m2], [other1, other2], b_value)

    # Return the full conjecture object (not the conclusion directly).
    return LinearConjecture(hypothesis, conclusion, symbol, touch)


In [104]:
# Example usage: Compute a valid upper bound on the independence number of connected
# graphs in terms of the order n and the minimum degree \delta.

# Call the function
make_upper_linear_conjecture_two_vars(df, "independence_number", "n", "minimum_degree", "connected")

If G is connected, then independence_number <= n -  minimum_degree

---

# Generating all Possible Inequalities on a Target Invariant

Now that we can conjecture on a target in terms of *two other invariants*, we write a Python function to iterate over all boolean properties and combinations of two numerical properties different than the target. This generates a Python list of conjecture objects on a given target invariant.

---

In [105]:
def make_all_upper_linear_conjectures_two_vars(df, target, others, properties):
    """
    Generates upper bound conjectures for all combinations of two invariants in the dataset.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the data.
    target : string
        The name of the target variable.
    others : list of strings
        The list of invariant names to consider for generating conjectures.
    properties : list of strings
        The list of boolean properties (hypotheses) to filter the dataset.

    Returns
    -------
    list
        A list of LinearConjecture objects representing the conjectures.
    """

    # Create conjectures for every pair of invariants in 'others' combined with each property
    conjectures = []

    # Iterate over all combinations of two invariants from 'others'
    for other1, other2 in combinations(others, 2):
        for prop in properties:
            # Ensure that neither of the 'other' invariants is equal to the target
            if other1 != target and other2 != target:
                # Generate the conjecture for this combination of two invariants
                conjecture = make_upper_linear_conjecture_two_vars(df, target, other1, other2, hyp=prop)
                conjectures.append(conjecture)
    return conjectures


def make_all_lower_linear_conjectures_two_vars(df, target, others, properties):
    # Create conjectures for every pair of invariants in 'others' combined with each property
    conjectures = []

    # Iterate over all combinations of two invariants from 'others'
    for other1, other2 in combinations(others, 2):
        for prop in properties:
            # Ensure that neither of the 'other' invariants is equal to the target
            if other1 != target and other2 != target:
                # Generate the conjecture for this combination of two invariants
                conjecture = make_lower_linear_conjecture_two_vars(df, target, other1, other2, hyp=prop)
                conjectures.append(conjecture)
    return conjectures




---

## Example: Generate all Possible Upper Bounds on the Independence Number $\alpha$

In the following code cell we generate upper bounds for the independence number of connected graphs, trees, connected and regular graphs, and connected and bipartite graphs in terms of combinations of two numerical columns in our tabular dataset.

---

In [106]:
# Define your target invariant, others (list of invariants), and properties (list of hypotheses)
target = "independence_number"
others = ["n", "minimum_degree", "matching_number", "maximum_degree"]
properties = ["connected", "connected and regular", "connected and bipartite"]

# Generate all possible conjectures for the target variable and invariant combinations
conjectures = make_all_upper_linear_conjectures_two_vars(df, target, others, properties)

# Sort the conjectures in nonincreasing order by the number of
# instances the inequality holds with equality.
conjectures.sort(key = lambda x: x.touch, reverse=True)

# Print the conjectures
for i, conj in enumerate(conjectures):
    print(f"Conjecture {i+1}. {conj}")
    print(f"The bound holds with equality on {conj.touch} graphs. \n ")

Conjecture 1. If G is connected, then independence_number <= n -  minimum_degree
The bound holds with equality on 7 graphs. 
 
Conjecture 2. If G is connected, then independence_number <= n -  matching_number
The bound holds with equality on 6 graphs. 
 
Conjecture 3. If G is connected and bipartite, then independence_number <= n -  matching_number
The bound holds with equality on 5 graphs. 
 
Conjecture 4. If G is connected and regular, then independence_number <= n -  minimum_degree
The bound holds with equality on 4 graphs. 
 
Conjecture 5. If G is connected and bipartite, then independence_number <= n -  minimum_degree
The bound holds with equality on 4 graphs. 
 
Conjecture 6. If G is connected and regular, then independence_number <= n -  maximum_degree
The bound holds with equality on 4 graphs. 
 
Conjecture 7. If G is connected and regular, then independence_number <= matching_number
The bound holds with equality on 3 graphs. 
 
Conjecture 8. If G is connected, then independenc

---

# Static-Dalmation Heuristic



---

In [107]:
def static_dalmatian(df, conjectures):
    # Start with the conjecture that has the highest touch number (first in the list).
    conj = conjectures[0]

    # Initialize the list of strong conjectures with the first conjecture.
    strong_conjectures = [conj]

    # Get the set of sharp graphs (i.e., graphs where the conjecture holds as equality) for the first conjecture.
    sharp_graphs = set(conj.get_sharp_graphs(df).index)

    # Iterate over the remaining conjectures in the list.
    for conj in conjectures[1:]:
        # Get the set of sharp graphs for the current conjecture.
        conj_sharp_graphs = set(conj.get_sharp_graphs(df).index)

        # Check if the current conjecture shares the same sharp graphs as any already selected strong conjecture.
        if any(conj_sharp_graphs == set(known.get_sharp_graphs(df).index) for known in strong_conjectures):
            # If it does, add the current conjecture to the list of strong conjectures.
            strong_conjectures.append(conj)

        # Otherwise, check if the current conjecture introduces new sharp graphs (graphs where the conjecture holds).
        elif conj_sharp_graphs - sharp_graphs != set():
            # If new sharp graphs are found, add the conjecture to the list.
            strong_conjectures.append(conj)

            # Update the set of sharp graphs to include the newly discovered sharp graphs.
            sharp_graphs = sharp_graphs.union(conj_sharp_graphs)

    # Return the list of strong, non-redundant conjectures.
    return strong_conjectures

In [108]:
# Sort the conjectures in nonincreasing order by the number of
# instances the inequality holds with equality.
conjectures.sort(key = lambda x: x.touch, reverse=True)

# Apply static-Dalmation.
conjs = static_dalmatian(df, conjectures)
print("TxGraffiti Conjectures the Following \n")
for i, conj in enumerate(conjs):
    print(f"Conjecture {i + 1}. {conj}")
    print(f"This bound is sharp on {conj.touch} graphs.")
    print()

TxGraffiti Conjectures the Following 

Conjecture 1. If G is connected, then independence_number <= n -  minimum_degree
This bound is sharp on 7 graphs.

Conjecture 2. If G is connected, then independence_number <= n -  matching_number
This bound is sharp on 6 graphs.



In [109]:
# The following function combines all of the conjecturing steps into a single
# simple Python function. This demonstrates that the conjecturing process can be
# expressed as a single algorithm - the WriteOnTheWallAlgorithm.
def write_on_the_wall(df, targets, invariant_names, property_names, use_dalmation=True):
    conjectures = []
    for target in targets:
        # Compute the upper bounds for the target.
        upper_conjectures = make_all_upper_linear_conjectures_two_vars(df, target, invariant_names, property_names)

        # # Compute the lower bounds for the target.
        lower_conjectures = make_all_lower_linear_conjectures_two_vars(df, target, invariant_names, property_names)

        # Sort both upper and lower bounds by touch number.
        upper_conjectures.sort(key = lambda x: x.touch, reverse=True)
        lower_conjectures.sort(key = lambda x: x.touch, reverse=True)

        # Apply static-Dalmation if requested. Note, this may drastically reduce
        # the number of conjectures presented. Provide use_dalmation=False to not
        # implement this.
        if use_dalmation:
            upper_conjectures = static_dalmatian(df, upper_conjectures)
            lower_conjectures = static_dalmatian(df, lower_conjectures)
        conjectures += upper_conjectures + lower_conjectures

    # Sort the combined conjectures by touch number.
    conjectures.sort(key = lambda x: x.touch, reverse=True)
    return conjectures

In [110]:
# Example usage: Formulate conjectures on both the independence number and matching number.
invariants = ["n", "minimum_degree", "matching_number", "maximum_degree", "independence_number"]
properties = ["connected", "connected and regular", "connected and bipartite"]
conjectures = write_on_the_wall(df, ["independence_number", "matching_number"], invariants, properties)


print("TxGraffiti Conjectures the Following \n")
for i, conj in enumerate(conjectures):
    print(f"Conjecture {i + 1}. {conj}")
    print(f"This bound is sharp on {conj.touch} graphs.")
    print()

TxGraffiti Conjectures the Following 

Conjecture 1. If G is connected, then independence_number <= n -  minimum_degree
This bound is sharp on 7 graphs.

Conjecture 2. If G is connected, then independence_number <= n -  matching_number
This bound is sharp on 6 graphs.

Conjecture 3. If G is connected, then independence_number >= - minimum_degree + maximum_degree + 1
This bound is sharp on 6 graphs.

Conjecture 4. If G is connected and bipartite, then independence_number >= n -  matching_number
This bound is sharp on 5 graphs.

Conjecture 5. If G is connected, then matching_number <= 1/2 * n
This bound is sharp on 5 graphs.

Conjecture 6. If G is connected, then matching_number <= 1/2 * n
This bound is sharp on 5 graphs.

Conjecture 7. If G is connected and bipartite, then matching_number <= n -  independence_number
This bound is sharp on 5 graphs.

Conjecture 8. If G is connected and bipartite, then matching_number >= n -  independence_number
This bound is sharp on 5 graphs.

Conjectur